# Package 

In [19]:
%load_ext autoreload
%autoreload 2
import sys
import logging
sys.path.append('../')
import os
from utils import *

from gensim.models import Word2Vec
import polars as pl

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Config 

In [13]:
w2v_model_dir = '../model_training/w2v.model'
test_data_dir = '.'
task = 'task1'


# Load model 

In [76]:
w2vec = Word2Vec.load(w2v_model_dir)

In [26]:
test_sessions = read_test_data(task, test_data_dir=test_data_dir)
test_sessions.sample(5)

,prev_items,locale
135025,"[B09KCHLWH4, B09KCHLWH4, B09F5V3CDN, B08MC3BQK...",JP
184024,"[B08R71STV6, B0BJCXF6QK, B0B9V3B882, B08R71STV...",JP
309425,"[B0007VTA5S, B01N7W8FRS, B08S8RHJTN]",UK
313773,"[B09D8QR3HS, B09D8QR3HS, B07DD5YHMH, B07DC5PPF...",UK
258407,"[B097DNKW7L, B09YMB42RM, B097D5JQVH, B097D84HQW]",UK


In [39]:
test_sessions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316971 entries, 0 to 316970
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   prev_items  316971 non-null  object
 1   locale      316971 non-null  object
dtypes: object(2)
memory usage: 4.8+ MB


In [27]:
# test_sessions['prev_items'] = test_sessions.apply(lambda row: process_item_lst(row), axis=1)

In [104]:
df = test_sessions.sample(1000)

In [105]:
for col in df.columns:
    df[col] = df[col].astype(str)

In [106]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 305876 to 237644
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   prev_items  1000 non-null   object
 1   locale      1000 non-null   object
dtypes: object(2)
memory usage: 23.4+ KB


In [108]:
def get_rec(prev_items):
    # print(prev_items)
    res = [ele.replace('[', '').replace(']', '').replace('\n', '').replace("'", '').replace(' ', '') for ele in prev_items.split(' ')]
    
    try:
        similarity_dic = w2vec.wv.most_similar(positive=prev_items, topn=100)
        res = [item for item, simi in similarity_dic] 
        # print(err)
    except Exception as e:
        print(e)
    
    return res

pl_df = pl.from_dataframe(df)
pl_df = (
    pl_df
        .with_column(pl.col('prev_items').apply(lambda row: get_rec(row)).alias('next_item_prediction'))
)
result_df = pl_df.to_pandas()

In [109]:
# pl_df

In [112]:
result_df.head()

,prev_items,locale,next_item_prediction
0,"['B0BCSSGR43', 'B09ZYT2MQY']",UK,"[B0BCSSGR43,, B09ZYT2MQY]"
1,"['B08FC3QNBV', 'B09TW38KVZ', 'B0B1CZJZK8', 'B0...",UK,"[B08FC3QNBV,, B09TW38KVZ,, B0B1CZJZK8,, B09CTT..."
2,"['B0BLTWKKKG', 'B07SLHM55V', 'B014P55MN0']",JP,"[B0BLTWKKKG,, B07SLHM55V,, B014P55MN0]"
3,"['B07VDBFRZW', 'B08TQM4F53', 'B07VDBFRZW', 'B0...",JP,"[B07VDBFRZW,, B08TQM4F53,, B07VDBFRZW,, B07QCB..."
4,"['B00F5JMDHE', 'B08SJNJLPQ', 'B07W5Z3YVQ', 'B0...",JP,"[B00F5JMDHE,, B08SJNJLPQ,, B07W5Z3YVQ,, B07W5Z..."
